# EXPLORACIÓN INTEGRAL DE DATOS CON PYTHON
[Pedro Marcelino](http://pmarcelino.com) - Febrero 2017

Otros Kernels: [Análisis de datos y extracción de características con Python
](https://www.kaggle.com/pmarcelino/data-analysis-and-feature-extraction-with-python)

----------

<b>'Lo más difícil en la vida es conocerse a uno mismo'</b>

Esta cita pertenece a Tales de Mileto. Thales fue un filósofo, matemático y astrónomo griego/fenicio, que es reconocido como el primer individuo en la civilización occidental conocido por haber entretenido y comprometido con el pensamiento científico (fuente: https://en.wikipedia.org/wiki/Thales)

No diría que conocer sus datos es lo más difícil en la ciencia de datos, pero requiere mucho tiempo. Por lo tanto, es fácil pasar por alto este paso inicial y saltar demasiado pronto al agua.

Así que traté de aprender a nadar antes de saltar al agua. Basado en [Hair et al. (2013)](https://amzn.to/2JuDmvo), capítulo 'Examinando sus datos', hice todo lo posible para seguir un análisis completo, pero no exhaustivo, de los datos. Estoy lejos de reportar un estudio riguroso en este kernel, pero espero que pueda ser útil para la comunidad, así que comparto cómo apliqué algunos de esos principios de análisis de datos a este problema.

A pesar de los nombres extraños que le di a los capítulos, lo que estamos haciendo en este kernel es algo así como:

1. <b>Comprender el problema</b>. Veremos cada variable y haremos un análisis filosófico sobre su significado e importancia para este problema.
2. <b>Estudio univariable</b>. Solo nos centraremos en la variable dependiente ('Precio de venta') e intentaremos saber un poco más al respecto.
3. <b>Estudio multivariante</b>. Intentaremos entender cómo se relacionan la variable dependiente y las variables independientes.
4. <b>Limpieza básica</b>. Limpiaremos el conjunto de datos y manejaremos los datos faltantes, los valores atípicos y las variables categóricas.
5. <b>Supuestos de prueba</b>. Verificaremos si nuestros datos cumplen con los supuestos requeridos por la mayoría de las técnicas multivariadas.

¡Ahora es el momento de divertirse!

In [ ]:
#invite people for the Kaggle party
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
#traer los seis paquetes
df_train = pd.read_csv('../input/train.csv')

In [ ]:
#revisar la decoración
df_train.columns

# 1. Entonces... ¿Qué podemos esperar?

Para comprender nuestros datos, podemos observar cada variable y tratar de comprender su significado y relevancia para este problema. Sé que esto lleva mucho tiempo, pero nos dará el sabor de nuestro conjunto de datos.

Para tener algo de disciplina en nuestro análisis, podemos crear una hoja de cálculo de Excel con las siguientes columnas:
* <b>Variable</b> - Nombre de la variable.
* <b>Tipo</b> - Identificación del tipo de las variables. Hay dos valores posibles para este campo: 'numérico' o 'categórico'. Por 'numérico' nos referimos a variables cuyos valores son números, y por 'categóricos' nos referimos a variables cuyos valores son categorías.
* <b>Segmento</b> - Identificación del segmento de las variables. Podemos definir tres posibles segmentos: edificio, espacio o ubicación. Cuando decimos 'edificio', nos referimos a una variable que se relaciona con las características físicas del edificio (por ejemplo, 'OverallQual'). Cuando decimos 'espacio', nos referimos a una variable que informa las propiedades de espacio de la casa (por ejemplo, 'TotalBsmtSF'). Finalmente, cuando decimos una 'ubicación', nos referimos a una variable que da información sobre el lugar donde se encuentra la casa (por ejemplo, 'Barrio').
* <b>Expectativa</b> - Nuestra expectativa sobre la influencia de la variable en 'SalePrice'. Podemos usar una escala categórica con 'Alto', 'Medio' y 'Bajo' como valores posibles.
* <b>Conclusión</b>: nuestras conclusiones sobre la importancia de la variable, después de dar un vistazo rápido a los datos. Podemos mantener la misma escala categórica que en 'Expectativa'.
* <b>Comentarios</b>: cualquier comentario general que se nos haya ocurrido.

Si bien 'Tipo' y 'Segmento' son solo para una posible referencia futura, la columna 'Expectativa' es importante porque nos ayudará a desarrollar un 'sexto sentido'. Para llenar esta columna, debemos leer la descripción de todas las variables y, una por una, preguntarnos:

* ¿Pensamos en esta variable cuando estamos comprando una casa? (por ejemplo, cuando pensamos en la casa de nuestros sueños, ¿nos importa su 'tipo de revestimiento de mampostería'?).
* Si es así, ¿qué importancia tendría esta variable? (por ejemplo, ¿cuál es el impacto de tener material 'Excelente' en el exterior en lugar de 'Deficiente'? ¿Y de tener 'Excelente' en lugar de 'Bueno'?).
* ¿Esta información ya está descrita en alguna otra variable? (por ejemplo, si 'LandContour' da la planitud de la propiedad, ¿realmente necesitamos saber el 'LandSlope'?).

Después de este ejercicio abrumador, podemos filtrar la hoja de cálculo y mirar cuidadosamente las variables con 'Alta' 'Expectativa'. Luego, podemos precipitarnos en algunos diagramas de dispersión entre esas variables y 'Precio de venta', completando la columna 'Conclusión', que es solo la corrección de nuestras expectativas.

Pasé por este proceso y llegué a la conclusión de que las siguientes variables pueden desempeñar un papel importante en este problema:

* OverallQual (que es una variable que no me gusta porque no sé cómo se calculó; un ejercicio divertido sería predecir 'OverallQual' usando todas las demás variables disponibles).
* Año de construcción.
* TotalBsmtSF.
* GrLivÁrea.

Terminé con dos variables de 'construcción' ('OverallQual' y 'YearBuilt') y dos variables de 'espacio' ('TotalBsmtSF' y 'GrLivArea'). Esto puede ser un poco inesperado ya que va en contra del mantra inmobiliario de que todo lo que importa es 'ubicación, ubicación y ubicación'. Es posible que este rápido proceso de examen de datos fuera un poco duro para las variables categóricas. Por ejemplo, esperaba que la variable 'Barrio' fuera más relevante, pero después del examen de datos terminé excluyéndola. Quizás esto esté relacionado con el uso de diagramas de dispersión en lugar de diagramas de caja, que son más adecuados para la visualización de variables categóricas. La forma en que visualizamos los datos a menudo influye en nuestras conclusiones.

Sin embargo, el punto principal de este ejercicio fue pensar un poco sobre nuestros datos y expectativas, por lo que creo que logramos nuestro objetivo. Ahora es el momento de 'un poco menos de conversación, un poco más de acción, por favor'. ¡Vamos a <b>sacudirlo!</b>

# 2. Lo primero es lo primero: analizar 'SalePrice'
​
'SalePrice' es la razón de nuestra búsqueda. Es como cuando vamos a una fiesta. Siempre tenemos una razón para estar ahí. Por lo general, las mujeres son esa razón. (descargo de responsabilidad: adáptalo a hombres, baile o alcohol, según tus preferencias)
​
Usando la analogía de las mujeres, construyamos una pequeña historia, la historia de 'Cómo conocimos a 'SalePrice'.
​
*Todo empezó en nuestra fiesta Kaggle, cuando buscábamos pareja de baile. Después de un rato buscando en la pista de baile, vimos a una chica, cerca de la barra, usando zapatos de baile. Esa es una señal de que ella está allí para bailar. Pasamos mucho tiempo haciendo modelos predictivos y participando en competencias de análisis, por lo que hablar con las chicas no es uno de nuestros superpoderes. Aun así, lo intentamos:*
​
*'¡Hola, soy Kaggly! ¿Y tú? 'Precio de venta'? ¡Que hermoso nombre! Conoces 'SalePrice', ¿podrías darme algunos datos sobre ti? Acabo de desarrollar un modelo para calcular la probabilidad de una relación exitosa entre dos personas. ¡Me gustaría aplicarlo a nosotros!'*

In [ ]:
#resumen de estadísticas descriptivas
df_train['SalePrice'].describe()

*'Muy bien... Parece que su precio mínimo es mayor que cero. ¡Excelente! ¡No tienes uno de esos rasgos personales que destruirían mi modelo! Tienes alguna foto que me puedas enviar? No sé... ¿tú en la playa... o tal vez una selfie en el gimnasio?'*

In [ ]:

#histogram
sns.distplot(df_train['SalePrice']);

*'¡Ay! Veo que usas maquillaje marinero cuando sales... ¡Qué elegante! También veo que tú:*

* *<b>Desviarse de la distribución normal.</b>*
* *<b>Tener un sesgo positivo apreciable.</b>*
* *<b>Mostrar picos.</b>*

* ¡Esto se está poniendo interesante! 'SalePrice', ¿podría darme las medidas de su cuerpo?'*

In [ ]:
#sesgo y curtosis
print("Skewness: %f" % df_train['SalePrice'].skew())
print("Kurtosis: %f" % df_train['SalePrice'].kurt())

*'¡Asombroso! Si mi calculadora de amor es correcta, nuestra probabilidad de éxito es 97.834657%. ¡Creo que deberíamos encontrarnos de nuevo! Por favor, conserve mi número y llámeme si está libre el próximo viernes. ¡Hasta luego, cocodrilo!'*

# 'SalePrice', sus amigos y sus intereses

*Es sabiduría militar elegir el terreno donde pelearás. Tan pronto como 'SalePrice' se fue, fuimos a Facebook. Sí, ahora esto se está poniendo serio. Tenga en cuenta que esto no es acoso. Es solo una investigación intensa de un individuo, si sabes a lo que me refiero.*

*Según su perfil, tenemos algunos amigos en común. Además de Chuck Norris, ambos conocemos 'GrLivArea' y 'TotalBsmtSF'. Además, también tenemos intereses comunes como 'OverallQual' y 'YearBuilt'. ¡Esto parece prometedor!*

*Para sacar el máximo provecho de nuestra investigación, comenzaremos mirando detenidamente los perfiles de nuestros amigos comunes y luego nos centraremos en nuestros intereses comunes.*

### Relación con variables numéricas

In [ ]:
#scatter plot grlivarea/saleprice
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

*Hmmm... Parece que 'SalePrice' y 'GrLivArea' son realmente viejos amigos, con una relación <b>lineal.</b>*

*¿Y qué hay de 'TotalBsmtSF'?*

In [ ]:
#scatter plot totalbsmtsf/saleprice
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

*'TotalBsmtSF' también es un gran amigo de 'SalePrice' ¡pero esta parece una relación mucho más emocional! Todo va bien y de repente, en una <b>fuerte reacción lineal (¿exponencial?)</b>, todo cambia. Además, está claro que a veces 'TotalBsmtSF' se cierra por sí mismo y no da crédito a 'SalePrice'.*

### Relación con características categóricas

In [ ]:
#box plot overallqual/saleprice
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);

*Como todas las chicas guapas, 'SalePrice' disfruta de 'OverallQual'. Nota personal: considere si McDonald's es adecuado para la primera cita.*

In [ ]:
var = 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);

*Aunque no es una tendencia fuerte, diría que 'SalePrice' es más propenso a gastar más dinero en cosas nuevas que en reliquias viejas.*
​
<b>Nota</b>: no sabemos si 'SalePrice' está en precios constantes. Los precios constantes tratan de eliminar el efecto de la inflación. Si 'SalePrice' no está en precios constantes, debería estarlo, para que los precios sean comparables a lo largo de los años.

### En resumen

Historias aparte, podemos concluir que:

* 'GrLivArea' y 'TotalBsmtSF' parecen estar linealmente relacionados con 'SalePrice'. Ambas relaciones son positivas, lo que significa que a medida que aumenta una variable, la otra también aumenta. En el caso de 'TotalBsmtSF', podemos ver que la pendiente de la relación lineal es particularmente alta.
* 'OverallQual' y 'YearBuilt' también parecen estar relacionados con 'SalePrice'. La relación parece ser más fuerte en el caso de 'OverallQual', donde el diagrama de caja muestra cómo aumentan los precios de venta con la calidad general.

Acabamos de analizar cuatro variables, pero hay muchas otras que deberíamos analizar. El truco aquí parece ser la elección de las características correctas (selección de características) y no la definición de relaciones complejas entre ellas (ingeniería de características).

Dicho esto, separemos el trigo de la paja.

# 3. Mantén la calma y trabaja inteligentemente

Hasta ahora solo hemos seguido nuestra intuición y analizado las variables que pensamos que son importantes. A pesar de nuestros esfuerzos por dar un carácter objetivo a nuestro análisis, debemos decir que nuestro punto de partida fue subjetivo.

Como ingeniero, no me siento cómodo con este enfoque. Toda mi educación se centró en desarrollar una mente disciplinada, capaz de resistir los vientos de la subjetividad. Hay una razón para eso. Trate de ser subjetivo en la ingeniería estructural y verá que la física hace que las cosas se derrumben. puede doler

Entonces, superemos la inercia y hagamos un análisis más objetivo.

### La 'sopa de plasma'

'Al principio no había nada excepto una sopa de plasma. Lo que se sabe de estos breves momentos en el tiempo, al comienzo de nuestro estudio de la cosmología, es en gran medida conjetura. Sin embargo, la ciencia ha ideado un bosquejo de lo que probablemente sucedió, basado en lo que se sabe sobre el universo hoy. (fuente: http://umich.edu/~gs265/bigbang.htm)

Para explorar el universo, comenzaremos con algunas recetas prácticas para darle sentido a nuestra 'sopa de plasma':
* Matriz de correlación (estilo mapa de calor).
* Matriz de correlación 'SalePrice' (estilo de mapa de calor ampliado).
* Gráficos de dispersión entre las variables más correlacionadas (muévase como el estilo de Jagger).

#### Matriz de correlación (estilo mapa de calor)

In [ ]:
#correlation matrix
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

En mi opinión, este mapa de calor es la mejor manera de obtener una visión general rápida de nuestra "sopa de plasma" y sus relaciones. (¡Gracias @seaborn!)

A primera vista, hay dos cuadrados de color rojo que me llaman la atención. El primero se refiere a las variables 'TotalBsmtSF' y '1stFlrSF', y el segundo se refiere a las variables 'Garage*X*'. Ambos casos muestran cuán significativa es la correlación entre estas variables. En realidad, esta correlación es tan fuerte que puede indicar una situación de multicolinealidad. Si pensamos en estas variables, podemos concluir que dan casi la misma información por lo que realmente se produce la multicolinealidad. Los mapas de calor son geniales para detectar este tipo de situaciones y en problemas dominados por la selección de características, como el nuestro, son una herramienta esencial.

Otra cosa que me llamó la atención fueron las correlaciones 'SalePrice'. Podemos ver nuestros conocidos 'GrLivArea', 'TotalBsmtSF' y 'OverallQual' diciendo un gran '¡Hola!', pero también podemos ver muchas otras variables que deben tenerse en cuenta. Eso es lo que haremos a continuación.

#### 'SalePrice' correlation matrix (zoomed heatmap style)

In [ ]:
#matriz de correlación de precios de venta
k = 10 #número de variables para el mapa de calor
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

Según nuestra bola de cristal, estas son las variables más correlacionadas con 'Precio de venta'. Mis pensamientos sobre esto:
​
* 'OverallQual', 'GrLivArea' y 'TotalBsmtSF' están fuertemente correlacionados con 'SalePrice'. ¡Controlar!
* 'GarageCars' y 'GarageArea' también son algunas de las variables más fuertemente correlacionadas. Sin embargo, como discutimos en el último subpunto, la cantidad de automóviles que caben en el garaje es una consecuencia del área del garaje. 'GarageCars' y 'GarageArea' son como hermanos gemelos. Nunca podrás distinguirlos. Por lo tanto, solo necesitamos una de estas variables en nuestro análisis (podemos mantener 'GarageCars' ya que su correlación con 'SalePrice' es mayor).
* 'TotalBsmtSF' y '1stFloor' también parecen ser hermanos gemelos. Podemos mantener 'TotalBsmtSF' solo para decir que nuestra primera suposición fue correcta (vuelva a leer 'Entonces... ¿Qué podemos esperar?').
* 'Baño Completo'?? ¿En realidad?
* 'TotRmsAbvGrd' y 'GrLivArea', hermanos gemelos nuevamente. ¿Es este conjunto de datos de Chernobyl?
* Ah... 'YearBuilt'... Parece que 'YearBuilt' está ligeramente correlacionado con 'SalePrice'. Honestamente, me asusta pensar en 'YearBuilt' porque empiezo a sentir que deberíamos hacer un poco de análisis de series de tiempo para hacerlo bien. Te dejo esto como tarea.
​
Procedamos a los diagramas de dispersión.

#### Gráficos de dispersión entre 'SalePrice' y variables correlacionadas (mover como estilo Jagger)

Prepárate para lo que estás a punto de ver. Debo confesar que la primera vez que vi estos diagramas de dispersión ¡quedé totalmente impresionado! Tanta información en tan poco espacio... Es simplemente increíble. Una vez más, ¡gracias @seaborn! ¡Me haces 'moverme como Jagger'!

In [ ]:
#scatterplot
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols], size = 2.5)
plt.show();

Aunque ya conocemos algunas de las figuras principales, este mega diagrama de dispersión nos da una idea razonable sobre las relaciones de las variables.

Una de las figuras que nos puede parecer interesante es la que hay entre 'TotalBsmtSF' y 'GrLiveArea'. En esta figura podemos ver los puntos dibujando una línea lineal, que casi actúa como un borde. Tiene mucho sentido que la mayoría de los puntos permanezcan debajo de esa línea. Las áreas del sótano pueden ser iguales al área de vivienda sobre el suelo, pero no se espera un área de sótano más grande que el área de vivienda sobre el suelo (a menos que esté tratando de comprar un búnker).

La trama relativa a 'SalePrice' y 'YearBuilt' también puede hacernos pensar. En la parte inferior de la 'nube de puntos', vemos lo que casi parece ser una tímida función exponencial (sea creativo). También podemos ver esta misma tendencia en el límite superior de la 'nube de puntos' (sé aún más creativo). Además, observe cómo el conjunto de puntos con respecto a los últimos años tiende a mantenerse por encima de este límite (solo quería decir que los precios están aumentando más rápido ahora).

Ok, suficiente de la prueba de Rorschach por ahora. Avancemos hacia lo que falta: ¡datos faltantes!

# 4. Datos faltantes

Preguntas importantes al pensar en datos faltantes:

* ¿Qué tan prevalentes son los datos que faltan?
* ¿Los datos que faltan son aleatorios o tienen un patrón?

La respuesta a estas preguntas es importante por razones prácticas porque la falta de datos puede implicar una reducción del tamaño de la muestra. Esto puede impedirnos continuar con el análisis. Además, desde una perspectiva sustantiva, debemos asegurarnos de que el proceso de datos faltantes no esté sesgado y oculte una verdad inconveniente.

In [ ]:
#datos perdidos
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Analicemos esto para entender cómo manejar los datos que faltan.

Consideraremos que cuando falta más del 15% de los datos, debemos eliminar la variable correspondiente y pretender que nunca existió. Esto quiere decir que no intentaremos ningún truco para rellenar los datos que faltan en estos casos. De acuerdo con esto, hay un conjunto de variables (por ejemplo, 'PoolQC', 'MiscFeature', 'Alley', etc.) que debemos eliminar. El punto es: ¿perderemos estos datos? No me parece. Ninguna de estas variables parece ser muy importante, ya que la mayoría de ellas no son aspectos en los que pensamos a la hora de comprar una casa (¿quizás por eso faltan datos?). Además, mirando más de cerca las variables, podríamos decir que variables como 'PoolQC', 'MiscFeature' y 'FireplaceQu' son fuertes candidatas para valores atípicos, por lo que estaremos encantados de eliminarlas.

En lo que respecta a los casos restantes, podemos ver que las variables 'Garaje*X*' tienen el mismo número de datos faltantes. Apuesto a que faltan datos se refiere al mismo conjunto de observaciones (aunque no lo comprobaré; es solo el 5 % y no deberíamos gastar 20 $ en problemas de 5 $). Dado que la información más importante sobre los garajes está expresada por 'GarageCars' y considerando que solo estamos hablando de un 5% de datos faltantes, eliminaré las variables 'Garage*X*' mencionadas. La misma lógica se aplica a las variables 'Bsmt*X*'.

Respecto a 'MasVnrArea' y 'MasVnrType', podemos considerar que estas variables no son imprescindibles. Además, tienen una fuerte correlación con 'YearBuilt' y 'OverallQual', que ya se han considerado. Así, no perderemos información si borramos 'MasVnrArea' y 'MasVnrType'.

Finalmente, tenemos una observación que falta en 'Electrical'. Dado que es solo una observación, eliminaremos esta observación y conservaremos la variable.

En resumen, para manejar los datos faltantes, eliminaremos todas las variables con datos faltantes, excepto la variable 'Electrical'. En 'Eléctrico' simplemente eliminaremos la observación con datos faltantes.

In [ ]:
#tratar con datos faltantes
df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train.isnull().sum().max() #just checking that there's no missing data missing...

# Fuera mentirosos!

Los valores atípicos también son algo de lo que debemos ser conscientes. ¿Por qué? Porque los valores atípicos pueden afectar notablemente a nuestros modelos y pueden ser una valiosa fuente de información, brindándonos información sobre comportamientos específicos.

Los valores atípicos son un tema complejo y merecen más atención. Aquí, solo haremos un análisis rápido a través de la desviación estándar de 'SalePrice' y un conjunto de diagramas de dispersión.

### Análisis univariado

La principal preocupación aquí es establecer un umbral que defina una observación como un valor atípico. Para hacerlo, estandarizaremos los datos. En este contexto, la estandarización de datos significa convertir valores de datos para que tengan una media de 0 y una desviación estándar de 1.

In [ ]:
#estandarización de datos
saleprice_scaled = StandardScaler().fit_transform(df_train['SalePrice'][:,np.newaxis]);
low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
high_range= saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]
print('outer range (low) of the distribution:')
print(low_range)
print('\nouter range (high) of the distribution:')
print(high_range)

Cómo luce 'SalePrice' con su nueva ropa:

* Los valores de rango bajo son similares y no muy alejados de 0.
* Los valores de rango alto están lejos de 0 y los valores de 7.algo están realmente fuera de rango.

Por ahora, no consideraremos ninguno de estos valores como un valor atípico, pero debemos tener cuidado con esos dos valores de 7.algo.

### Análisis bivariado

Ya conocemos de memoria los siguientes diagramas de dispersión. Sin embargo, cuando miramos las cosas desde una nueva perspectiva, siempre hay algo que descubrir. Como dijo Alan Kay, 'un cambio de perspectiva vale 80 puntos de coeficiente intelectual'.

In [ ]:
#análisis bivariado saleprice/grlivarea
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

Lo que se ha revelado:

* Los dos valores con 'GrLivArea' más grande parecen extraños y no siguen a la multitud. Podemos especular por qué sucede esto. Tal vez se refieran a zona agrícola y eso podría explicar el bajo precio. No estoy seguro de esto, pero estoy bastante seguro de que estos dos puntos no son representativos del caso típico. Por lo tanto, los definiremos como valores atípicos y los eliminaremos.
* Las dos observaciones en la parte superior de la gráfica son esas 7.algunas observaciones con las que dijimos que debemos tener cuidado. Parecen dos casos especiales, sin embargo parecen estar siguiendo la tendencia. Por esa razón, los mantendremos.

In [ ]:
#borrado de puntos
df_train.sort_values(by = 'GrLivArea', ascending = False)[:2]
df_train = df_train.drop(df_train[df_train['Id'] == 1299].index)
df_train = df_train.drop(df_train[df_train['Id'] == 524].index)

In [ ]:
#bivariate analysis saleprice/grlivarea
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

Podemos sentirnos tentados a eliminar algunas observaciones (por ejemplo, TotalBsmtSF > 3000) pero supongo que no vale la pena. Podemos vivir con eso, así que no haremos nada.

# 5. Ponerse duro

En la novela de Ayn Rand, 'La rebelión de Atlas', hay una pregunta que se repite a menudo: ¿quién es John Galt? Una gran parte del libro trata sobre la búsqueda para descubrir la respuesta a esta pregunta.

Me siento Randian ahora. ¿Quién es 'SalePrice'?

La respuesta a esta pregunta radica en probar los supuestos que subyacen a las bases estadísticas para el análisis multivariante. Ya hicimos una limpieza de datos y descubrimos mucho sobre 'SalePrice'. Ahora es el momento de profundizar y comprender cómo 'SalePrice' cumple con los supuestos estadísticos que nos permiten aplicar técnicas multivariadas.

Según [Hair et al. (2013)](https://amzn.to/2uC3j9p), se deben probar cuatro supuestos:

* <b>Normalidad</b>: cuando hablamos de normalidad, lo que queremos decir es que los datos deben verse como una distribución normal. Esto es importante porque varias pruebas estadísticas se basan en esto (por ejemplo, estadísticas t). En este ejercicio, solo verificaremos la normalidad univariante para 'SalePrice' (que es un enfoque limitado). Recuerde que la normalidad univariante no asegura la normalidad multivariante (que es lo que nos gustaría tener), pero ayuda. Otro detalle a tener en cuenta es que en muestras grandes (>200 observaciones) la normalidad no es un problema. Sin embargo, si resolvemos la normalidad, evitamos muchos otros problemas (por ejemplo, heteroscedacidad), por lo que esa es la razón principal por la que estamos haciendo este análisis.

* <b>Homocedasticidad</b>: solo espero haberlo escrito bien. La homocedasticidad se refiere a la 'suposición de que la(s) variable(s) dependiente(s) exhibe(n) niveles iguales de varianza en el rango de la(s) variable(s) predictora(s)' [(Hair et al., 2013)](https://amzn.to/2uC3j9p). La homocedasticidad es deseable porque queremos que el término de error sea el mismo en todos los valores de las variables independientes.

* <b>Linealidad</b>: la forma más común de evaluar la linealidad es examinar diagramas de dispersión y buscar patrones lineales. Si los patrones no son lineales, valdría la pena explorar las transformaciones de datos. Sin embargo, no entraremos en esto porque la mayoría de los diagramas de dispersión que hemos visto parecen tener relaciones lineales.

* <b>Ausencia de errores correlacionados</b>: los errores correlacionados, como sugiere la definición, ocurren cuando un error está correlacionado con otro. Por ejemplo, si un error positivo genera sistemáticamente un error negativo, significa que existe una relación entre estas variables. Esto ocurre a menudo en series de tiempo, donde algunos patrones están relacionados con el tiempo. Tampoco entraremos en esto. Sin embargo, si detecta algo, intente agregar una variable que pueda explicar el efecto que está obteniendo. Esa es la solución más común para errores correlacionados.

¿Qué crees que diría Elvis sobre esta larga explicación? ¿'Un poco menos de conversación, un poco más de acción, por favor'? Probablemente... Por cierto, ¿sabes cuál fue el último gran éxito de Elvis?

(...)

El suelo del baño.

### En busca de la normalidad

El punto aquí es probar 'SalePrice' de una manera muy sencilla. Lo haremos prestando atención a:

* <b>Histograma</b> - Curtosis y asimetría.
* <b>Gráfica de probabilidad normal</b>: la distribución de datos debe seguir de cerca la diagonal que representa la distribución normal.

In [ ]:
#histograma y gráfico de probabilidad normal
sns.distplot(df_train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

Ok, 'SalePrice' no es normal. Muestra 'pico', asimetría positiva y no sigue la línea diagonal.

Pero no todo está perdido. Una simple transformación de datos puede resolver el problema. Esta es una de las cosas asombrosas que puede aprender en los libros de estadística: en caso de asimetría positiva, las transformaciones logarítmicas generalmente funcionan bien. Cuando descubrí esto, me sentí como un estudiante de Hogwarts descubriendo un nuevo hechizo genial.

*¡Avada kedavra!*

In [ ]:
#histograma y gráfico de probabilidad normal
df_train['SalePrice'] = np.log(df_train['SalePrice'])

In [ ]:
#histograma transformado y diagrama de probabilidad normal
sns.distplot(df_train['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

¡Hecho! Veamos qué está pasando con 'GrLivArea'.

In [ ]:
#histograma y gráfico de probabilidad normal
sns.distplot(df_train['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

Sabe a sesgo... *¡Abre kedavra!*

In [ ]:
#transformación de datos
df_train['GrLivArea'] = np.log(df_train['GrLivArea'])

In [ ]:
#histograma transformado y diagrama de probabilidad normal
sns.distplot(df_train['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

Siguiente por favor...

In [ ]:
#histograma y gráfico de probabilidad normal
sns.distplot(df_train['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train['TotalBsmtSF'], plot=plt)

Ok, ahora estamos tratando con el gran jefe. ¿Qué tenemos aquí?

* Algo que, en general, presenta asimetría.
* Un número significativo de observaciones con valor cero (viviendas sin sótano).
* Un gran problema porque el valor cero no nos permite hacer transformaciones logarítmicas.

Para aplicar aquí una transformación logarítmica, crearemos una variable que pueda tener el efecto de tener o no tener sótano (variable binaria). Luego, haremos una transformación logarítmica a todas las observaciones distintas de cero, ignorando aquellas con valor cero. De esta forma podemos transformar los datos, sin perder el efecto de tener o no base.

No estoy seguro de si este enfoque es correcto. Simplemente me pareció correcto. Eso es lo que yo llamo 'ingeniería de alto riesgo'.

In [ ]:
#crear columna para nueva variable (una es suficiente porque es una característica categórica binaria)
#si area>0 obtiene 1, para area==0 obtiene 0
df_train['HasBsmt'] = pd.Series(len(df_train['TotalBsmtSF']), index=df_train.index)
df_train['HasBsmt'] = 0 
df_train.loc[df_train['TotalBsmtSF']>0,'HasBsmt'] = 1

In [ ]:
#transformar datos
df_train.loc[df_train['HasBsmt']==1,'TotalBsmtSF'] = np.log(df_train['TotalBsmtSF'])

In [ ]:
#histogram and normal probability plot
sns.distplot(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)

### En la búsqueda de escribir 'homocedasticidad' en el primer intento

El mejor enfoque para probar la homocedasticidad de dos variables métricas es gráficamente. Las desviaciones de una dispersión igual se muestran mediante formas como conos (pequeña dispersión en un lado del gráfico, gran dispersión en el lado opuesto) o rombos (una gran cantidad de puntos en el centro de la distribución).

Empezando por 'SalePrice' y 'GrLivArea'...

In [ ]:
#scatter plot
plt.scatter(df_train['GrLivArea'], df_train['SalePrice']);

Las versiones anteriores de este diagrama de dispersión (anteriores a las transformaciones de registros) tenían una forma cónica (regrese y marque 'Gráficos de dispersión entre 'Precio de venta' y variables correlacionadas (mover como estilo Jagger)'). Como puede ver, el gráfico de dispersión actual ya no tiene forma cónica. ¡Ese es el poder de la normalidad! Solo asegurando la normalidad en algunas variables, resolvimos el problema de homocedasticidad.

Ahora vamos a comprobar 'SalePrice' con 'TotalBsmtSF'.

In [ ]:
#scatter plot
plt.scatter(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], df_train[df_train['TotalBsmtSF']>0]['SalePrice']);

Podemos decir que, en general, 'SalePrice' presenta niveles iguales de varianza en el rango de 'TotalBsmtSF'. ¡Fresco!

# Por último, pero no menos importante, variables ficticias

Modo fácil.

In [ ]:
#convertir variable categórica en ficticia
df_train = pd.get_dummies(df_train)

# Conclusión

¡Eso es todo! Llegamos al final de nuestro ejercicio.

A lo largo de este núcleo ponemos en práctica muchas de las estrategias propuestas por [Hair et al. (2013)](https://amzn.to/2uC3j9p). Filosofamos sobre las variables, analizamos 'Precio de venta' solo y con las variables más correlacionadas, tratamos con datos faltantes y valores atípicos, probamos algunos de los supuestos estadísticos fundamentales e incluso transformamos variables categóricas en variables ficticias. Eso es mucho trabajo que Python nos ayudó a hacer más fácil.

Pero la búsqueda no ha terminado. Recuerda que nuestra historia se detuvo en la investigación de Facebook. Ahora es el momento de llamar a 'SalePrice' e invitarla a cenar. Trate de predecir su comportamiento. ¿Crees que es una chica que disfruta de los enfoques de regresión lineal regularizada? ¿O crees que prefiere los métodos de conjunto? ¿O tal vez algo más?

Depende de ti averiguarlo.

# <b>Referencias</b>
* [Mi blog](http://pmarcelino.com)
* [Mis otros kernels](https://www.kaggle.com/pmarcelino/data-analysis-and-feature-extraction-with-python)
* [Hair et al., 2013, Análisis de datos multivariados, 7.ª edición](https://amzn.to/2JuDmvo)

# Agradecimientos

Gracias a [João Rico](https://www.linkedin.com/in/joaomiguelrico/) por leer borradores de esto.

# ¿Querer aprender más?

Estoy creando un curso interactivo de 5 semanas basado en cohortes para personas que desean aprender los conceptos básicos de Machine Learning en Python y dejar de preguntarse si pueden comenzar una carrera en este campo. [Complete esta breve encuesta](https://tinyurl.com/learn-machine-learning) para ingresar a la lista de espera y recibir un descuento especial por reserva anticipada.